In [7]:
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchdiffeq import odeint_adjoint as odeint
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [8]:
def load_data_normalize(obs_dim, datafilepath):
    datafilepath = 'C:/Users/shiny/Documents/NeuralODE_HumanGait/Humangaitdata.npy'
    data = np.load(datafilepath)
    traj_tot = np.load(datafilepath).reshape(72, 1500, obs_dim)
    traj_tot = traj_tot[:,150:1350,:]
    data = data[:, 300:1300, :]
    data = data.reshape(72, 1000, obs_dim)
    noise_std = 0.2

    orig_trajs = np.zeros((data.shape[0],data.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = data[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            orig_trajs[i,:,j] = (trajs - trajs_tot.mean()) / trajs_tot.std()
            
    #samp_trajs += npr.randn(*samp_trajs.shape) * noise_std #add noise

    return orig_trajs


In [9]:
def load_data_normalize(obs_dim, datafilepath):
    datafilepath = 'C:/Users/shiny/Documents/NeuralODE_HumanGait/Humangaitdata.npy'
    data = np.load(datafilepath)
    traj_tot = np.load(datafilepath).reshape(72, 1500, obs_dim)
    traj_tot = traj_tot[:,50:1450,:]
    data = data[:, 50:1450, :]
    data = data.reshape(72, 1400, obs_dim)
    noise_std = 0.2

    orig_trajs = np.zeros((data.shape[0],data.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = data[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            orig_trajs[i,:,j] = (trajs - trajs_tot.mean()) / trajs_tot.std()
            
    #samp_trajs += npr.randn(*samp_trajs.shape) * noise_std #add noise

    return orig_trajs


In [10]:
def takenembedding(data, tau, k, obs_dim, device):
    data_TE = np.zeros((data.shape[0], data.shape[1]-tau*k, data.shape[2]), dtype = object)
    
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            for t in range(data.shape[1]-tau*k):
                data_TE[i,t,j] = data[i, t:t+tau*k+1, j][::tau][::-1]
                
    data_TE = np.array(data_TE.tolist())
    data_TE = data_TE.reshape(data_TE.shape[0],data_TE.shape[1],obs_dim*(k+1))
    
    return data_TE
    

In [2]:
npr.seed(42)
tau = 18
k = 5

obs_dim = 6
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

orig_trajs = load_data_normalize(obs_dim, 'C:/Users/shiny/Documents/NeuralODE_HumanGait/Humangaitdata.npy')

orig_trajs_TE = takenembedding(orig_trajs, tau, k, obs_dim, device)

NameError: name 'load_data_normalize' is not defined

In [11]:
npr.seed(42)
tau = 18
k = 5

obs_dim = 6
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

orig_trajs = load_data_normalize(obs_dim, 'C:/Users/shiny/Documents/NeuralODE_HumanGait/Humangaitdata.npy')

orig_trajs_TE = takenembedding(orig_trajs, tau, k, obs_dim, device)

In [37]:
fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(15, 9))
axes = axes.flatten()
gen_index = 10
dataset_value = 58
times_index = 0
deriv_index=0
ts_pos_combined = np.linspace(0, 0.25*gen_index, num=50*gen_index) 
for i, ax in enumerate(axes):
    ax.plot(ts_pos_combined[times_index:times_index+50*gen_index], orig_trajs[dataset_value,times_index:times_index+50*gen_index, i])
    ax.set_ylim(-2.5, 2.5)

        
plot_name = 'lookingatwhole_dataset{}.png'.format(dataset_value)
plt.savefig(plot_name, dpi=500)
plt.close()

## Data for comparing two data RNN VS ODE

In [51]:
orig_trajs = np.load('C:/Users/shiny/Documents/NeuralODE_HumanGait/Humangaitdata.npy')

In [52]:
RNN_data = np.load('C:/Users/shiny/Documents/NeuralODE_HumanGait/UNIT_512_LB_499_All_preds_with_smallest_noise.npy')

In [65]:
fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(15, 9))
axes = axes.flatten()
gen_index = 30
dataset_value = 4
times_index = 0
deriv_index=0
plot_index = 1500
tot_index = 80
ts_pos_combined = np.linspace(0, 0.25*tot_index, num=50*tot_index) 
for i, ax in enumerate(axes):
    ax.scatter(ts_pos_combined[times_index:times_index+50*gen_index], second_orig_trajs[dataset_value,times_index:times_index+50*gen_index, i], s = 5)
    ax.plot(ts_pos_combined[times_index:times_index+50*tot_index], RNN_data[dataset_value,times_index:times_index+50*tot_index, i],'r')
    
plot_name = 'Comparingtwodatasets_{}.png'.format(dataset_value)
plt.savefig(plot_name, dpi=500)
plt.close()    

In [66]:
for i in range(12):
    fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(15, 9))
    axes = axes.flatten()
    gen_index = 30
    dataset_value = i
    times_index = 0
    deriv_index=0
    plot_index = 1500
    tot_index = 100
    ts_pos_combined = np.linspace(0, 0.25*tot_index, num=50*tot_index) 
    for i, ax in enumerate(axes):
        ax.scatter(ts_pos_combined[times_index:times_index+50*gen_index], second_orig_trajs[dataset_value,times_index:times_index+50*gen_index, i], s = 5)
        ax.plot(ts_pos_combined[times_index:times_index+50*tot_index], RNN_data[dataset_value,times_index:times_index+50*tot_index, i],'r')

    plot_name = 'Comparingtwodatasets_{}.png'.format(dataset_value)
    plt.savefig(plot_name, dpi=500)
    plt.close()  

## Adding Speed

In [40]:
import scipy.io
speed = scipy.io.loadmat('SpeedLabels.mat')
speed_label = speed['SpeedTrainTrials']

In [4]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k6.npy')

In [41]:
X = np.zeros((orig_trajs_TE.shape[0],orig_trajs_TE.shape[1],1))

for i in range(orig_trajs_TE.shape[0]):
    X[i, :, 0] = speed_label[i]
    

In [42]:
orig_trajs_TE = np.concatenate((orig_trajs_TE, X), axis=2)

In [43]:
np.save('orig_trajs_TE_tau18k5_raw.npy',orig_trajs_TE)

In [4]:
obs_dim = 6
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

orig_trajs = load_data_normalize(obs_dim, 'C:/Users/shiny/Documents/NeuralODE_HumanGait/Humangaitdata.npy')

In [8]:
X = np.zeros((orig_trajs.shape[0],orig_trajs.shape[1],1))

for i in range(orig_trajs.shape[0]):
    X[i, :, 0] = speed_label[i]/100
    
orig_trajs_speed = np.concatenate((orig_trajs, X), axis=2)

In [13]:
orig_trajs_speed.shape

(72, 1000, 7)

In [10]:
np.save('orig_trajs_speed.npy', orig_trajs_speed)

In [4]:
npr.seed(42)
tau = 18
k = 5

obs_dim = 6
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Creating Maximal LIST

In [13]:
orig_trajs_TE.shape

(72, 1310, 36)

In [14]:
np.save('orig_trajs_TE_tau18k6_timestep500.npy', orig_trajs_TE)

In [18]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k6_timestep500.npy')
orig_trajs_TE = orig_trajs_TE[:, :1300, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72, 1300, obs_dim*(k+1))


samp_trajs_TE = np.zeros((72*8, 500, obs_dim*(k+1)))
samp_trajs_TE = torch.from_numpy(samp_trajs_TE).float().to(device)
samplepoint = [0, 100, 200 ,300, 400 ,500, 600, 700]
for j in range(72):
    for i in range(8):
        samp_trajs_TE[8*j+i,:, :] = orig_trajs_TE[j, samplepoint[i]:samplepoint[i]+500,:]
    
    
samplepoint_val_TE = np.zeros((72*1, 500, obs_dim*(k+1)))
samplepoint_val_TE = torch.from_numpy(samplepoint_val_TE).float().to(device)
samplepoint_val = [800]    
for j in range(72):
    for i in range(1):
        samplepoint_val_TE[1*j+i,:, :] = orig_trajs_TE[j, samplepoint_val[i]:samplepoint_val[i]+500,:]
        
        
torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k5_timestep500.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k5_timestep500.pt')

In [19]:
samp_trajs_TE.shape

torch.Size([576, 500, 36])

In [44]:

orig_trajs_TE = np.load('orig_trajs_TE_tau18k5_raw.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72, 900, obs_dim*(k+1)+1)


samp_trajs_TE = np.zeros((72*6, 300, obs_dim*(k+1)+1))
samp_trajs_TE = torch.from_numpy(samp_trajs_TE).float().to(device)
samplepoint = [0, 100, 200 ,300, 400 ,500]
for j in range(72):
    for i in range(6):
        samp_trajs_TE[6*j+i,:, :] = orig_trajs_TE[j, samplepoint[i]:samplepoint[i]+300,:]
    
    
    
samp_trajs_val_TE = orig_trajs_TE[:, 600:900, :]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k5_raw_rlong.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k5_raw_rlong.pt')

In [16]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72, 900, obs_dim*(k+1)+1)


samp_trajs_TE = np.zeros((72*6, 300, obs_dim*(k+1)+1))
samp_trajs_TE = torch.from_numpy(samp_trajs_TE).float().to(device)
samplepoint = [0, 100, 200 ,300, 400 ,500]
for j in range(72):
    for i in range(6):
        samp_trajs_TE[6*j+i,:, :] = orig_trajs_TE[j, samplepoint[i]:samplepoint[i]+300,:]
    
    
    
samp_trajs_val_TE = orig_trajs_TE[:, 600:900, :]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k5_rlong.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k5_rlong.pt')

In [45]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72, 900, obs_dim*(k+1)+1)


samp_trajs_TE = np.zeros((72*6, 300, obs_dim*(k+1)+1))
samp_trajs_TE = torch.from_numpy(samp_trajs_TE).float().to(device)
samplepoint = [0, 100, 200 ,300, 400 ,500]
for j in range(72):
    for i in range(6):
        samp_trajs_TE[6*j+i,:, :] = orig_trajs_TE[j, samplepoint[i]:samplepoint[i]+300,:]
    
    
    
samp_trajs_val_TE = orig_trajs_TE[:, 600:900, :]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k5_rlong.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k5_rlong.pt')

In [ ]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
person = 0
orig_trajs_TE = orig_trajs_TE[person*6:(person+1)*6, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(6*3, 300, obs_dim*(k+1)+1)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:15]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[15:18]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_person{}_tau18k5_rlong.pt'.format(person))
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_person{}_tau18k5_rlong.pt'.format(person))

In [6]:
## MAKE EACH PERSON A SAMPLE!

for i in range(12):
    person = i
    orig_trajs_TE = np.load('orig_trajs_TE_tau18k6.npy')
    orig_trajs_TE = orig_trajs_TE[person*6:(person+1)*6, :900, :]
    orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(6, 900, obs_dim*(k+1))
    
    samp_trajs_TE = np.zeros((6*6, 300, obs_dim*(k+1)))
    samp_trajs_TE = torch.from_numpy(samp_trajs_TE).float().to(device)
    samplepoint = [0, 100, 200 ,300, 400 ,500]
    for j in range(6):
        for k in range(6):
            samp_trajs_TE[6*j+k,:, :] = orig_trajs_TE[j, samplepoint[k]:samplepoint[k]+300,:]
    
    samp_trajs_val_TE = orig_trajs_TE[:, 600:900, :]

    
    torch.save(samp_trajs_TE, 'samp_trajs_TE_person{}_tau18k6_rlong.pt'.format(person))
    torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_person{}_tau18k6_rlong.pt'.format(person))

In [67]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k6.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72, 900, obs_dim*(k+1))


samp_trajs_TE = np.zeros((72*6, 300, obs_dim*(k+1)))
samp_trajs_TE = torch.from_numpy(samp_trajs_TE).float().to(device)
samplepoint = [0, 100, 200 , 300,  400 , 500]
for j in range(72):
    for i in range(len(samplepoint)):
        samp_trajs_TE[6*j+i,:, :] = orig_trajs_TE[j, samplepoint[i]:samplepoint[i]+300,:]
        
        
samp_trajs_val_TE = np.zeros((72*1, 300, obs_dim*(k+1)))
samp_trajs_val_TE = torch.from_numpy(samp_trajs_val_TE).float().to(device)
samplepoint_val = [600]

for j in range(72):
    for i in range(len(samplepoint_val)):
        samp_trajs_val_TE[len(samplepoint_val)*j+i,:, :] = orig_trajs_TE[j, samplepoint_val[i]:samplepoint_val[i]+300,:]    
    

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k6_rlong.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k6_rlong.pt')

In [4]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k6.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72*6, 150, obs_dim*(k+1))

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:360]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[360:432]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k6_long.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k6_long.pt')

In [12]:
orig_trajs_TE = np.load('orig_trajs_speed.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72*3, 300, obs_dim+1)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:36*5]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[180:216]]

torch.save(samp_trajs_TE, 'samp_trajs_speed_rlong.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_speed_rlong.pt')

In [14]:
orig_trajs_TE = np.load('orig_trajs_speed.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72*9, 100, obs_dim+1)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:36*15]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[540:72*9]]

torch.save(samp_trajs_TE, 'samp_trajs_speed.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_speed.pt')

In [8]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
person = 0
orig_trajs_TE = orig_trajs_TE[person*6:(person+1)*6, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(6*9, 100, obs_dim*(k+1)+1)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:45]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[45:54]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_person{}_tau18k5.pt'.format(person))
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_person{}_tau18k5.pt'.format(person))

In [5]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
person = 0
orig_trajs_TE = orig_trajs_TE[person*6:(person+1)*6, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(6*3, 300, obs_dim*(k+1)+1)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:15]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[15:18]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_person{}_tau18k5_rlong.pt'.format(person))
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_person{}_tau18k5_rlong.pt'.format(person))

In [5]:
for i in range(12):
    person = i
    orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
    orig_trajs_TE = orig_trajs_TE[person*6:(person+1)*6, :900, :]
    orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(6*9, 100, obs_dim*(k+1)+1)
    
    perm_index = torch.randperm(orig_trajs_TE.shape[0])
    samp_trajs_TE = orig_trajs_TE[perm_index[:45]]
    samp_trajs_val_TE = orig_trajs_TE[perm_index[45:54]]
    
    torch.save(samp_trajs_TE, 'samp_trajs_TE_person{}_tau18k5.pt'.format(person))
    torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_person{}_tau18k5.pt'.format(person))

In [ ]:
for i in range(12):
    person = i
    orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
    orig_trajs_TE = orig_trajs_TE[person*6:(person+1)*6, :900, :]
    orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(6*3, 300, obs_dim*(k+1)+1)
    
    perm_index = torch.randperm(orig_trajs_TE.shape[0])
    samp_trajs_TE = orig_trajs_TE[perm_index[:15]]
    samp_trajs_val_TE = orig_trajs_TE[perm_index[15:18]]
    
    torch.save(samp_trajs_TE, 'samp_trajs_TE_person{}_tau18k5_rlong.pt'.format(person))
    torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_person{}_tau18k5_rlong.pt'.format(person))

In [10]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k5.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72*3, 300, obs_dim*(k+1)+1)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:36*5]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[180:216]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k5_rlong.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k5_rlong.pt')

In [43]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k6.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72*6, 150, obs_dim*(k+1))

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:360]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[360:432]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k6_long.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k6_long.pt')

In [45]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k6.npy')
orig_trajs_TE = orig_trajs_TE[:, :900, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72*18, 50, obs_dim*(k+1))

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:900]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[900:1200]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k6_short.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k6_short.pt')

In [53]:
orig_trajs_TE = np.load('orig_trajs_TE_tau18k6.npy')
orig_trajs_TE = orig_trajs_TE[:, :875, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(72*5, 175, obs_dim*(k+1))

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:300]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[300:360]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_tau18k6_longer.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_tau18k6_longer.pt')

In [54]:
samp_trajs_TE.shape

torch.Size([300, 175, 36])

In [49]:
torch.mean(a,1)

tensor([-0.2307, -0.8338,  0.0083,  0.4351])

In [52]:
torch.mean(a)

tensor(-0.1553)

In [6]:
orig_trajs_TE = np.load('orig_trajs_TE_tau6k9.npy')
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(203*34*4, 50, obs_dim*(k+1)+2)


perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:3600]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[4000:4600]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_nonoise_tau6k9.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_nonoise_tau6k9.pt')

In [4]:
orig_trajs_TE = np.load('orig_trajs_TE_tau6k9.npy')
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(203*34*4, 50, obs_dim*(k+1)+2)

perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:15000]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[21000:24000]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_nonoise_tau6k9_long.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_nonoise_tau6k9_long.pt')

In [3]:
orig_trajs_TE = np.load('orig_trajs_TE_tau6k3.npy')
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(203*34*2, 100, obs_dim*(k+1)+2)


perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:1500]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[4000:4200]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_nonoise_tau6k3_timestep100.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_nonoise_tau6k3_timestep100.pt')

In [4]:
orig_trajs_TE = np.load('orig_trajs_TE_tau6k3.npy')
orig_trajs_TE = orig_trajs_TE.reshape(203, 34*200, obs_dim*(k+1)+2)[:, :2000, :]
orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(203*10*2, 100, obs_dim*(k+1)+2)


perm_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_TE = orig_trajs_TE[perm_index[:1500]]
samp_trajs_val_TE = orig_trajs_TE[perm_index[1500:1800]]

torch.save(samp_trajs_TE, 'samp_trajs_TE_nonoise_tau6k3_timestep100.pt')
torch.save(samp_trajs_val_TE, 'samp_trajs_val_TE_nonoise_tau6k3_timestep100.pt')

In [ ]:
orig_trajs_TE = np.load('orig_trajs_TE_tau6k3.npy')

orig_trajs_TE = torch.from_numpy(orig_trajs_TE).float().to(device).reshape(203*34, 200, obs_dim*(k+1)+2)

val_index = torch.randperm(orig_trajs_TE.shape[0])
samp_trajs_val_TE = orig_trajs_TE[val_index[:1000]]

torch.save(samp_trajs_val_TE, 'samp_trajs_TE_4cycle_tau6k3.pt')

In [ ]:
samp_trajs_TE = torch.load('samp_trajs_TE_nonoise_tau6k3.pt')
samp_trajs_val_TE = torch.load('samp_trajs_val_TE_nonoise_tau6k3.pt')

tau = 6
k = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

samp_ts = np.linspace(0, 0.25, num=50)
samp_ts = torch.from_numpy(samp_ts).float().to(device)

orig_trajs_TE = np.load('orig_trajs_TE_tau6k3.npy')
orig_trajs_TE = orig_trajs_TE.reshape(203, 200*34, 31*(k+1)+2)
samp_trajs_TE_test = orig_trajs_TE[:, :50, :]
samp_trajs_TE_test = torch.from_numpy(samp_trajs_TE_test).float().to(device).reshape(203, 50, 31*(k+1)+2)